In [ ]:
from crewai import Crew, Task, Agent
from langchain_openai import ChatOpenAI
import os
from personal_search.tools.books import LibraryTool, YoutubeTool
import warnings
from pydantic import BaseModel, Field
from typing import List
from IPython.display import display, Markdown, Latex
from dotenv import load_dotenv

load_dotenv()

warnings.filterwarnings("ignore")

model_name = os.environ.get("OPENAI_MODEL_NAME", "gpt-3.5-turbo")

llm = ChatOpenAI(model=model_name, temperature=0)

class TopicoEmLivros(BaseModel):
	"""Modelo de Tópico em Livros"""
	titulo: str = Field(..., description="Título do tópico")
	encontrado: bool = Field(..., description="O tópico foi encontrado em livros?")
	livros: List[str] = Field(..., description="Livros que tratam do tópico")
	citacoes: List[str] = Field(..., description="Citações de livros sobre o tópico")
	tratamento: str = Field(..., description="Como o tópico é tratado nos livros")
	fontes: List[str] = Field(..., description="Fontes do tópico")

In [ ]:
# abrir arquivo yaml como dict
def load_yaml(file_path):
    import yaml
    with open(file_path, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

agents_config = load_yaml("src/personal_search/config/agents.yaml")
tasks_config = load_yaml("src/personal_search/config/tasks.yaml")

In [ ]:
researcher = Agent(
	config=agents_config['book_researcher'],
    tools=[LibraryTool()],
    verbose=True,
    llm=llm,
	allow_delegation=False,
)

research = Task(
	config=tasks_config['topic_in_books'],
    agent=researcher,
    output_pydantic=TopicoEmLivros,
)

crew = Crew(
  agents=[researcher],
  tasks=[research],
  verbose=2,
  memory=True,
  llm=llm,
  full_output=True,
  planning=True,
)

input_text  = "O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas."

inputs = {
    "topic": input_text
}
result = crew.kickoff(inputs=inputs)

In [ ]:
def extract_markdown(result_dict, key):
    markdown = ''
    # verifica se o valor é uma lista
    if isinstance(result_dict[key], list):
        for item in result_dict[key]:
            markdown += f"- {item}\n"
    else:
        markdown += f"{result_dict[key]}\n\n"
    return markdown

def split_text(text):
    text = text.split('_')
    text = ' '.join(text)
    return text.title()

def print_pydantic(result):
    markdown = ''
    keys = result.pydantic.dict().keys()
    result_dict = result.pydantic.dict()
    for key in keys:
        if key == 'titulo':
            markdown += f"# {extract_markdown(result_dict,key)}\n"
        elif key == 'classificacao':
            markdown += f"**Classificação:** {extract_markdown(result_dict,key)}\n"
        else:
            markdown += f"## {split_text(key)}\n"
            markdown += extract_markdown(result_dict, key)
    return markdown

Markdown(print_pydantic(result))

In [ ]:
Markdown(result.raw)